In [2]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/tests
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


In [13]:
from src.utils.embedding_utils import load_df_from_csv_with_embedding
from src.utils.config import OUTPUT_PATH

final_entities=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_entities_with_embeddings_by_chatgpt4o.csv",['entity_embedding'])
final_entities

,entity_name,entity_type,description,entity_id,corpus_id,entity_embedding
0,AMAZON,ORGANIZATION,Amazon is a retail giant known for its extensi...,0,0,"[0.025474460795521736, -0.019272109493613243, ..."
1,CYBER MONDAY,EVENT,Cyber Monday is a significant shopping event t...,1,0,"[0.008677905425429344, -0.012837450951337814, ..."
2,BLACK FRIDAY,EVENT,Black Friday is a shopping holiday that takes ...,2,0,"[0.016857707872986794, -0.01078620832413435, -..."
3,"NOVEMBER 27, 2023",TIME,"November 27, 2023, is the date when Cyber Mond...",3,0,"[0.009768207557499409, -0.01335572637617588, -..."
4,AMAZON,ORGANIZATION,Amazon is a multinational technology company k...,4,1,"[0.029136504977941513, -0.007207231130450964, ..."
...,...,...,...,...,...,...
39353,ISRAEL-HAMAS WAR,EVENT,"A conflict involving Israel and Hamas, which h...",39353,4476,"[0.04042772203683853, -0.0035180281847715378, ..."
39354,EEA,LOCATION,The European Economic Area includes the Europe...,39354,4476,"[0.02691667526960373, -0.003410726087167859, -..."
39355,SWITZERLAND,LOCATION,A country in Europe that is part of the Europe...,39355,4476,"[0.015187944285571575, -0.011422696523368359, ..."
39356,DIGITAL SERVICES ACT,CONCEPT,The Digital Services Act is a regulation estab...,39356,4476,"[-0.010030446574091911, -0.01822492480278015, ..."


In [14]:
final_relationships=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_relationships_with_embeddings_by_chatgpt4o.csv",['relationship_embedding'])
final_relationships

,source_entity,target_entity,relationship_description,relationship_strength,source_entity_id,target_entity_id,corpus_id,relationship_embedding
0,AMAZON,CYBER MONDAY,Amazon is hosting numerous deals during Cyber ...,9,0,1,0,"[0.02443057671189308, -0.03799114748835564, -0..."
1,AMAZON,BLACK FRIDAY,Amazon began its Black Friday sale a week ahea...,8,0,2,0,"[0.054795797914266586, -0.031081540510058403, ..."
2,CYBER MONDAY,BLACK FRIDAY,"Cyber Monday follows Black Friday, with many d...",7,1,2,0,"[0.009650849737226963, -0.029358115047216415, ..."
3,"NOVEMBER 27, 2023",CYBER MONDAY,"November 27, 2023, is the date that marks the ...",10,3,1,0,"[0.007443748414516449, -0.01130700297653675, -..."
4,AMAZON,ECHO SHOW,"Amazon produces the Echo Show, integrating it ...",8,4,5,1,"[0.045543570071458817, -0.009272989816963673, ..."
...,...,...,...,...,...,...,...,...
29203,META,ISRAEL-HAMAS WAR,Meta's approach to content regulation is being...,6,39351,39353,4476,"[0.04234350472688675, -0.02529730089008808, -0..."
29204,META,DIGITAL SERVICES ACT,Meta is designated as a gatekeeper under the D...,8,39351,39356,4476,"[0.034957364201545715, -0.028356697410345078, ..."
29205,META,DIGITAL MARKETS ACT,Meta is also impacted by the Digital Markets A...,8,39351,39357,4476,"[0.010328758507966995, -0.016240786761045456, ..."
29206,EEA,META,"Meta serves users across the EEA, impacting it...",7,39354,39351,4476,"[0.034811727702617645, -0.027029171586036682, ..."


In [25]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# 假设你使用 Sentence-BERT 作为嵌入模型
model = SentenceTransformer('intfloat/multilingual-e5-large')

# 将问题转换为嵌入向量
def get_query_embedding(query):
    query_embedding = model.encode(query)
    return query_embedding

# 计算查询嵌入和实体嵌入之间的余弦相似度
def find_k_nearest_entities(query_embedding, final_entities, k=3):
    # 过滤掉嵌入为空的情况
    valid_entities = final_entities.dropna(subset=['entity_embedding'])
    # 确保所有嵌入向量的维度一致
    entity_embeddings = np.array([emb for emb in valid_entities['entity_embedding'].values if len(emb) == len(query_embedding)])
    similarities = cosine_similarity([query_embedding], entity_embeddings)[0]
    valid_entities = valid_entities.loc[valid_entities['entity_embedding'].apply(lambda x: len(x) == len(query_embedding))]
    valid_entities['similarity'] = similarities
    nearest_entities = valid_entities.sort_values(by='similarity', ascending=False).head(k)
    return nearest_entities

# 设置显示的最大列宽
pd.set_option('display.max_colwidth', 200)
# 获取查询嵌入
query = "Which company, known for its dominance in the e-reader space and for offering exclusive invite-only deals during sales events, faced a stock decline due to an antitrust lawsuit reported by 'The Sydney Morning Herald' and discussed by sellers in a 'Cnbc | World Business News Leader' article?"
query_embedding = get_query_embedding(query)

# 查询最相近的 k 个实体
k = 3
nearest_entities = find_k_nearest_entities(query_embedding, final_entities, k)
nearest_entities[['entity_name','description', 'similarity']]

,entity_name,description,similarity
14019,SYDNEY MORNING HERALD,Sydney Morning Herald is an Australian newspaper that alerted Beyond Blue about the privacy issue with TikTok.,0.810759
37837,KINDLE PAPERWHITE,"The Kindle Paperwhite is an e-reader developed by Amazon, often featured in sales events for discounted prices.",0.803172
17613,GOOGLE,Google is a technology company known for its search engine and various online services. The company is involved in an antitrust trial concerning its market dominance.,0.799831
